# NoSQL - AKA Other Database Structures

Credit to my colleagues Victor and Amber for much of this content!

## Is SQL not good enough for you??

SQL databases are Relational Databases built really for tabular data

Focus on being structured, which can be useful/safe but also restrictive

## Introducing "Not Only SQL" (aka NoSQL)

These are non-relational databases (sometimes called distributed databases)

AWS has a great resource describing NoSQL and how its different from SQL: https://aws.amazon.com/nosql/

### NoSQL Advantages :

- Don't need to know structure when you start
- Flexible (specific to database)
- Horizontally scalable: "Just spin-up a new server!"

(Why aren't these true for SQL databases? Let's discuss!)

# PokemonGo DB with Mongo DB 

Start by running `mongod` in your terminal - let's debug what problems we have all together!

For me, and anyone else running a MacOS with Catalina, we need to specify our `--dbpath` argument!

In [1]:
import pandas as pd
import pymongo

Note: we interact with MongoDB using pymongo - [here is their documentation](https://pymongo.readthedocs.io/en/stable/) (featuring tutorials and examples).

**Connect to Mongo, create a database and a collection.**

- Host: `'localhost'`
- Port: `27017` (how can we check this?)

In [33]:
client = pymongo.MongoClient(host="localhost", port=27017)
poke_db = client['pokemon_db']
poke_collection = poke_db['pokemon_collection']

**Create an entry and insert it into the collection.**

In [3]:
pikachu = {'name': 'Pikachu', 'primary_type': 'Electric', 'secondary_type': 'None', 'level':25}
#Now let's insert this data into our collection below
poke_collection.insert_one(pikachu)

**Run a query to find all items in the collection.**

In [4]:
[x for x in poke_collection.find()]

[{'_id': ObjectId('5f973d52bbb483ee1ee2e761'),
  'name': 'Pikachu',
  'primary_type': 'Electric',
  'secondary_type': 'None',
  'level': 25}]

**Insert multiple entries at once.**

In [6]:
charmander = {'name': 'Charmander', 'primary_type': 'Fire', 'secondary_type': 'None', 'level':12}
bulbasaur = {'name': 'Bulbasaur',  'primary_type': 'Grass', 'secondary_type': 'Poison', 'level':10}
squirtle = {'name': 'Squirtle',  'primary_type': 'Water', 'secondary_type': 'None', 'level':8}

starters_list = [charmander, bulbasaur, squirtle]

results = poke_collection.insert_many(starters_list)

**Query just the names of all the entered Pokemon without the IDs.**

In [11]:
[x for x in poke_collection.find({}, {'_id':0, 'name':1})]

[{'name': 'Pikachu'},
 {'name': 'Charmander'},
 {'name': 'Bulbasaur'},
 {'name': 'Squirtle'}]

**Query the Pokemon with a level less than 20. You may need the reference manual** [here](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors)

In [9]:
[x for x in poke_collection.find({'level': {'$lt':20}})]

[{'_id': ObjectId('5f973d98bbb483ee1ee2e762'),
  'name': 'Charmander',
  'primary_type': 'Fire',
  'secondary_type': 'None',
  'level': 12},
 {'_id': ObjectId('5f973d98bbb483ee1ee2e763'),
  'name': 'Bulbasaur',
  'primary_type': 'Grass',
  'secondary_type': 'Poison',
  'level': 10},
 {'_id': ObjectId('5f973d98bbb483ee1ee2e764'),
  'name': 'Squirtle',
  'primary_type': 'Water',
  'secondary_type': 'None',
  'level': 8}]

Now let's undo everything we've done!

1. **Delete Pikachu.** 


In [12]:
poke_collection.delete_one({'name':'Pikachu'})

In [13]:
# Check your work
[x for x in poke_collection.find()]

[{'_id': ObjectId('5f973d98bbb483ee1ee2e762'),
  'name': 'Charmander',
  'primary_type': 'Fire',
  'secondary_type': 'None',
  'level': 12},
 {'_id': ObjectId('5f973d98bbb483ee1ee2e763'),
  'name': 'Bulbasaur',
  'primary_type': 'Grass',
  'secondary_type': 'Poison',
  'level': 10},
 {'_id': ObjectId('5f973d98bbb483ee1ee2e764'),
  'name': 'Squirtle',
  'primary_type': 'Water',
  'secondary_type': 'None',
  'level': 8}]

2. **Delete all entries without a secondary type.**

In [14]:
poke_collection.delete_many({'secondary_type': 'None'})

In [15]:
# Check your work
[x for x in poke_collection.find()]

[{'_id': ObjectId('5f973d98bbb483ee1ee2e763'),
  'name': 'Bulbasaur',
  'primary_type': 'Grass',
  'secondary_type': 'Poison',
  'level': 10}]

3. **Delete the entire collection.**

In [16]:
poke_collection.drop()

In [17]:
# Check your work
[x for x in poke_collection.find()]

[]

In [18]:
poke_db.list_collection_names()

[]

## Additional Resources

https://university.mongodb.com/ - full set of courses on MongoDB, all free!
 - Start with [M001 - MongoDB Basics](https://university.mongodb.com/courses/M001/about) if you want a taste test not only in running some of these commands, but to get practice exploring how to connect and run these commands in their cloud interface

## API Introduction (if we have time!)

### The Many Use Cases For APIs

APIs can be used for many things - much more than just retrieving information. Twilio has an API that allows you to write a script to send text messages to people. GitHub has an API for creating new repositories. Many services have APIs allowing computers to automate tasks that a person might otherwise have to do through a website - whether uploading a photo to Flickr, searching for a company name in a state database, or getting a list of garbage collection times for a municipality.

### "Requests is the only Non-GMO HTTP library for Python, safe for human consumption."

> "Requests allows you to send organic, grass-fed HTTP/1.1 requests, without the need for manual labor."

Straight from the `requests` [documentation](https://pypi.org/project/requests/)

In [19]:
import requests

### Types of requests

We will mostly use GET requests in order to get data, but there are other options.

![CRUD image from ProDataMan Blog](http://blog.prodataman.com/wp-content/uploads/2018/09/CRUD-operations.jpg)

[Image Source](http://blog.prodataman.com/2018/09/19/crud-script-and-ssms-toolkit/)

That's right - CRUD summarizes the kinds of requests you can make with most APIs. 

Let's say you are looking at an API for a car rental company like Hertz or Zipcar - the following different requests could generate these different responses:

| Request               | Result                               | In CRUD Terms |
| --------------------- | ------------------------------------ | ------------- |
| GET /stores/          | User sees the list of stores         | Read          |
| GET /rentals/         | User sees the history of car rentals | Read          |
| POST /rentals/        | User rents a car                     | Create        |
| PUT /rentals/{id}/    | User changes destination store       | Update        |
| DELETE /rentals/{id}/ | User cancels the active car rental   | Delete        |


## Practice: The Pokemon API!

https://pokeapi.co/

Nice place to start because there's no login details or credentials to fuss with

For now, just want to explore grabbing the Types of different Pokemon.

In [20]:
base_path = 'https://pokeapi.co/api/v2/'

In [21]:
pokemon = 'charmander'

url = f'{base_path}/pokemon/{pokemon}'

response = requests.get(url)

In [22]:
response

<Response [200]>

In [23]:
response.text

'{"abilities":[{"ability":{"name":"blaze","url":"https://pokeapi.co/api/v2/ability/66/"},"is_hidden":false,"slot":1},{"ability":{"name":"solar-power","url":"https://pokeapi.co/api/v2/ability/94/"},"is_hidden":true,"slot":3}],"base_experience":62,"forms":[{"name":"charmander","url":"https://pokeapi.co/api/v2/pokemon-form/4/"}],"game_indices":[{"game_index":176,"version":{"name":"red","url":"https://pokeapi.co/api/v2/version/1/"}},{"game_index":176,"version":{"name":"blue","url":"https://pokeapi.co/api/v2/version/2/"}},{"game_index":176,"version":{"name":"yellow","url":"https://pokeapi.co/api/v2/version/3/"}},{"game_index":4,"version":{"name":"gold","url":"https://pokeapi.co/api/v2/version/4/"}},{"game_index":4,"version":{"name":"silver","url":"https://pokeapi.co/api/v2/version/5/"}},{"game_index":4,"version":{"name":"crystal","url":"https://pokeapi.co/api/v2/version/6/"}},{"game_index":4,"version":{"name":"ruby","url":"https://pokeapi.co/api/v2/version/7/"}},{"game_index":4,"version":{"

In [24]:
response.json().keys()

dict_keys(['abilities', 'base_experience', 'forms', 'game_indices', 'height', 'held_items', 'id', 'is_default', 'location_area_encounters', 'moves', 'name', 'order', 'species', 'sprites', 'stats', 'types', 'weight'])

In [30]:
response.json()['types']

[{'slot': 1,
  'type': {'name': 'fire', 'url': 'https://pokeapi.co/api/v2/type/10/'}}]

In [31]:
pokemon = 'bulbasaur'

url = f'{base_path}/pokemon/{pokemon}'

response2 = requests.get(url)

In [32]:
response2.json()['types']

[{'slot': 1,
  'type': {'name': 'grass', 'url': 'https://pokeapi.co/api/v2/type/12/'}},
 {'slot': 2,
  'type': {'name': 'poison', 'url': 'https://pokeapi.co/api/v2/type/4/'}}]

Type chart, for reference:

![Pokemon type chart, from pokemondb.net](https://img.pokemondb.net/images/typechart.png)

[Image Source](https://pokemondb.net/type)